In [1]:
import time
import datetime
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from data import example_to_tensor, normalize
from train import EarlyStopping
from utils import plot_slice, plot_animated_volume

print(f"Tensorflow: {tf.__version__}")
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

Tensorflow: 2.3.0


In [2]:
verbose_training = True
seed = 5
# Hyperparameters
epochs = 1000
learning_rate = 0.0001
patience = 5
batch_size = 4
test_num_samples = 10
validation_num_samples = 10
input_shape = (248, 128, 128, 1)  # downscale 4
# input_shape = (16, 128, 128, 1)  # downscale 4 (nii only)
# input_shape = (488, 256, 256, 1)  # downscale 2
# input_shape = (964, 512, 512, 1)  # original

In [3]:
# This is to allocate GPU memory only when needed

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# Use mixed precision float16/float32

policy = mixed_precision.Policy("mixed_float16")
# policy = mixed_precision.Policy('float32')
mixed_precision.set_policy(policy)
print("Compute dtype: %s" % policy.compute_dtype)
print("Variable dtype: %s" % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [5]:
data_dir = Path("data")
tfrecord_fnames = [
    str(p)
    for g in (
        data_dir.glob("tcia-0.25/*.tfrecord"),
        data_dir.glob("nrrd-0.25/*.tfrecord"),
        # data_dir.glob("nii-0.25/*.tfrecord"),
    )
    for p in g
]

full_dataset = tf.data.TFRecordDataset(tfrecord_fnames)
full_dataset = full_dataset.map(
    example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE
)
full_dataset = full_dataset.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE
)
full_dataset = full_dataset.map(
    lambda x: tf.expand_dims(x, axis=-1),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

In [6]:
dataset = full_dataset.shuffle(
    buffer_size=32, seed=seed, reshuffle_each_iteration=False
)

test_dataset = dataset.take(test_num_samples)
test_dataset = test_dataset.batch(1)
dataset = dataset.skip(test_num_samples)

val_dataset = dataset.take(validation_num_samples)
val_dataset = val_dataset.padded_batch(
    batch_size=batch_size, padded_shapes=input_shape,
)

train_dataset = dataset.skip(validation_num_samples)
train_dataset = train_dataset.padded_batch(
    batch_size=batch_size, padded_shapes=input_shape,
)
train_dataset = train_dataset.cache()  # must be called before shuffle
train_dataset = train_dataset.shuffle(buffer_size=64, reshuffle_each_iteration=True)
train_dataset = train_dataset.take(16)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
train_dataset

<PrefetchDataset shapes: (None, 248, 128, 128, 1), types: tf.float32>

In [7]:
def benchmark(dataset, num_epochs=4):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    tf.print("Execution time:", time.perf_counter() - start_time)


# benchmark(train_dataset)

In [8]:
def conv_block(x, filters, kernel_size=3, dropout_rate=0.1, pool_size=2):
    """
    - Convolution 3D
    - Selu activation
    - Dropout
    - Max pool 3D
    
    x is the input layer using the Keras Functional API
    """
    x = keras.layers.Conv3D(
        filters=filters,
        kernel_size=kernel_size,
        padding="same",
        kernel_initializer="lecun_normal",
        bias_initializer="lecun_normal",
        activation="selu",
    )(x)
    x = keras.layers.AlphaDropout(dropout_rate)(x)
    x = keras.layers.MaxPool3D(pool_size=pool_size)(x)
    return x

In [9]:
def deconv_block(x, filters, kernel_size=3, dropout_rate=0.1, pool_size=2):
    """
    - Up sampling 3D
    - Convolution 3D
    - Selu activation
    - Dropout
    
    x is the input layer using the Keras Functional  API
    """
    x = keras.layers.UpSampling3D(size=pool_size)(x)
    x = keras.layers.Conv3D(
        filters=filters,
        kernel_size=kernel_size,
        padding="same",
        kernel_initializer="lecun_normal",
        bias_initializer="lecun_normal",
        activation="selu",
    )(x)
    x = keras.layers.AlphaDropout(dropout_rate)(x)
    return x

In [10]:
encoder_inputs = keras.Input(input_shape, batch_size=batch_size)
x = conv_block(encoder_inputs, filters=16)
encoder_outputs = conv_block(x, filters=32)
encoder = keras.Model(encoder_inputs, encoder_outputs, name="encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(4, 248, 128, 128, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (4, 248, 128, 128, 16)    448       
_________________________________________________________________
alpha_dropout (AlphaDropout) (4, 248, 128, 128, 16)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (4, 124, 64, 64, 16)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (4, 124, 64, 64, 32)      13856     
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (4, 124, 64, 64, 32)      0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (4, 62, 32, 32, 32)       0   

In [11]:
decoder_inputs = keras.Input(encoder.output_shape[1:], batch_size=batch_size)
x = deconv_block(decoder_inputs, filters=32)
x = deconv_block(x, filters=16)
x = keras.layers.Dense(1)(x)
decoder_outputs = keras.layers.Activation("sigmoid", dtype="float32")(x)
decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(4, 62, 32, 32, 32)]     0         
_________________________________________________________________
up_sampling3d (UpSampling3D) (4, 124, 64, 64, 32)      0         
_________________________________________________________________
conv3d_2 (Conv3D)            (4, 124, 64, 64, 32)      27680     
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (4, 124, 64, 64, 32)      0         
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (4, 248, 128, 128, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (4, 248, 128, 128, 16)    13840     
_________________________________________________________________
alpha_dropout_3 (AlphaDropou (4, 248, 128, 128, 16)    0   

In [12]:
autoencoder = keras.models.Sequential([encoder, decoder], name="autoencoder")
# autoencoder.load_weights("models/autoencoder/20200924-235155/best_epoch_ckpt")
# autoencoder = keras.models.load_model("models/autoencoder/20200924-235155")

assert autoencoder.output_shape[1:] == input_shape
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (4, 62, 32, 32, 32)       14304     
_________________________________________________________________
decoder (Functional)         (4, 248, 128, 128, 1)     41537     
Total params: 55,841
Trainable params: 55,841
Non-trainable params: 0
_________________________________________________________________


In [13]:
loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(lr=learning_rate)
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale="dynamic")

In [ ]:
start_time = datetime.datetime.now()
start_time_str = start_time.strftime("%Y%m%d-%H%M%S")
log_dir = f"logs/experiments/{start_time_str}/"
model_dir = f"models/experiments/{start_time_str}/"
ckpt_dir = model_dir + "best_epoch_ckpt"
writer = tf.summary.create_file_writer(log_dir)

early_stopping = EarlyStopping(patience)

for epoch in tqdm(range(epochs), disable=False):

    ### TRAIN ###

    train_loss_metric = tf.keras.metrics.Mean("train_loss", dtype=tf.float32)
    for batch in train_dataset:
        with tf.GradientTape() as tape:
            predictions = autoencoder(batch)
            loss_value = loss_fn(predictions, batch)
            scaled_loss_value = optimizer.get_scaled_loss(loss_value)
        scaled_gradients = tape.gradient(scaled_loss_value, autoencoder.trainable_variables)
        gradients = optimizer.get_unscaled_gradients(scaled_gradients)
        optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))
        train_loss_metric.update_state(loss_value)
        with writer.as_default():
            for grad, param in zip(gradients, autoencoder.trainable_variables):
                tf.summary.histogram(param.name, param, step=epoch)
                # tf.summary.histogram(param.name + "/grad", grad, buckets=1, step=epoch)

    train_loss_mean = train_loss_metric.result()
    with writer.as_default():
        tf.summary.scalar("Training loss", train_loss_mean, step=epoch)
    train_loss_metric.reset_states()

    ### VALIDATION ###

    val_loss_metric = tf.keras.metrics.Mean("val_loss", dtype=tf.float32)
    for batch in val_dataset:
        predictions = autoencoder(batch, training=False)
        val_loss_metric.update_state(loss_fn(predictions, batch))

    val_loss_mean = val_loss_metric.result()
    with writer.as_default():
        tf.summary.scalar("Validation loss", val_loss_mean, step=epoch)
    val_loss_metric.reset_states()

    if verbose_training:
        print()
        print(f"Epoch : {epoch}")
        print(f"Training loss: {train_loss_mean}")
        print(f"Validation loss: {val_loss_mean}")

    ### EARLY STOPPING ###

    early_stopping.update(val_loss_mean)
    if early_stopping.early_stop:
        autoencoder.load_weights(ckpt_dir)
        break
    elif early_stopping.not_improving_epochs == 0:
        autoencoder.save_weights(ckpt_dir)

autoencoder.save(model_dir)

end_time = datetime.datetime.now()
training_time = str(end_time - start_time).split(".")[0]

with writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"batch size = {batch_size}; "
        f"patience = {patience}; "
        f"learning rate = {learning_rate}; "
        f"training time = {training_time}",
        step=0,
    )


Epoch : 0
Training loss: 0.04763423651456833
Validation loss: 0.011542127467691898

Epoch : 1
Training loss: 0.01523527130484581
Validation loss: 0.011322778649628162

Epoch : 2
Training loss: 0.015419526025652885
Validation loss: 0.011290603317320347

Epoch : 3
Training loss: 0.019479166716337204
Validation loss: 0.01126996148377657

Epoch : 4
Training loss: 0.012479502707719803
Validation loss: 0.011246838606894016

Epoch : 5
Training loss: 0.014211487025022507
Validation loss: 0.011207862757146358


In [ ]:
reload_ext tensorboard
%tensorboard --logdir=logs --bind_all

In [ ]:
original = next(iter(test_dataset.skip(0)))
encoder_out = autoencoder.layers[0](original)
decoder_out = autoencoder.layers[1](encoder_out)
batch_index = 0

In [ ]:
z_index = 5
fig, ax = plt.subplots(ncols=3)
plot_slice(original, batch_index, z_index, ax[0])
plot_slice(encoder_out, batch_index, encoder_out.shape[1] // 3, ax[1])
plot_slice(decoder_out, batch_index, z_index, ax[2])

In [ ]:
plot_animated_volume(original, batch_index)

In [ ]:
plot_animated_volume(encoder_out, batch_index, fps=10)

In [ ]:
plot_animated_volume(decoder_out, batch_index)